In [ ]:
!pip install --upgrade transformers datasets accelerate peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 947.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0


In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer
)
from peft import get_peft_model, LoraConfig, TaskType
from trl import SFTTrainer

In [ ]:
print("Step 3: Setting configuration...")
model_name = "google/gemma-2b"
dataset_name = "imdb"
# core configuration for LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,                                       # The rank of the LoRA matrices
    lora_alpha=16,                             # Scaling factor for LoRA matrices
    lora_dropout=0.1,                          # Dropout for LoRA layers
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none"                                # No bias parameters trained
)

Step 3: Setting configuration...


In [ ]:
print("Step 4: Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Tokenizer pad token set to EOS token.")

Step 4: Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
print("Step 5: Loading and preprocessing dataset...")
dataset = load_dataset(dataset_name)

def formatting_func(example):
    sentiment = 'positive' if example['label'] == 1 else 'negative'
    prompt_template = f"Review: {{text}}\nSentiment: {sentiment}{tokenizer.eos_token}"

    max_review_tokens = 512 - len(tokenizer("\nSentiment: " + sentiment + tokenizer.eos_token)['input_ids']) - 10
    inputs = tokenizer(example['text'], truncation=True, max_length=max_review_tokens)
    truncated_text = tokenizer.decode(inputs['input_ids'], skip_special_tokens=True)

    return prompt_template.format(text=truncated_text)

small_train_dataset = dataset["train"].shuffle(seed=42).select(range(500))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(100))
print("Dataset prepared.")

print("Step 6: Loading the base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
base_model.config.pad_token_id = tokenizer.pad_token_id

print("\n--- Model has been loaded. Now checking GPU memory usage. ---")
!nvidia-smi

print("Step 7: Applying LoRA to the model...")
lora_model = get_peft_model(base_model, lora_config)

print("\nTrainable parameters:")
lora_model.print_trainable_parameters()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("\nStep 8: Setting up training arguments...")
training_args = TrainingArguments(
    output_dir="./lora_imdb_classifier_results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

trainer = SFTTrainer(
    model=lora_model,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    peft_config=lora_config,
    formatting_func=formatting_func,
    args=training_args
)

print("Step 9: Starting training...")
trainer.train()
print("Training complete.")

print("\nStep 10: Running inference on test sentences...")
lora_model.to("cuda")
lora_model.eval()

positive_review = "This movie was absolutely incredible! The performances were brilliant and the story was captivating."
prompt = f"Review: {positive_review}\nSentiment:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    output = lora_model.generate(**inputs, max_new_tokens=10, do_sample=False)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
predicted_sentiment = generated_text.split("Sentiment:")[1].strip().split()[0]

print(f"\nReview: '{positive_review}'")
print(f"Predicted Sentiment: {predicted_sentiment}")

negative_review = "I was really let down by this film. The plot was predictable and the acting was wooden."
prompt = f"Review: {negative_review}\nSentiment:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    output = lora_model.generate(**inputs, max_new_tokens=10, do_sample=False)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
predicted_sentiment = generated_text.split("Sentiment:")[1].strip().split()[0]

print(f"\nReview: '{negative_review}'")
print(f"Predicted Sentiment: {predicted_sentiment}")

Step 5: Loading and preprocessing dataset...


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Step 9: Starting training...


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.84 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.87 GiB is free. Process 4528 has 11.87 GiB memory in use. Of the allocated memory 10.48 GiB is allocated by PyTorch, and 1.26 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)